# Подготовка окружения

In [1]:
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim

from transformers import AutoTokenizer, pipeline
from torch.utils.data import DataLoader, RandomSampler
from torch.serialization import add_safe_globals
from configs.config import Config
from src.data_utils import process_dataset, split_dataset
from src.next_token_dataset import TextDataset
from src.lstm_model import LSTMModel, count_parameters, generate_samples
from src.lstm_train import train_model
from src.eval_lstm import calculate_rouge_lstm
from src.eval_transformer_pipeline import calculate_rouge_transformer

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
IS_TEST = False # тестовый сценарий для отладки кода на слабой машине
config = Config(is_test=IS_TEST) # конфигурационные параметры

tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
tokenizer.pad_token = tokenizer.pad_token or tokenizer.eos_token

config.pad_token_id = tokenizer.pad_token_id or tokenizer.eos_token_id
config.pad_token = tokenizer.pad_token
config.vocab_size = len(tokenizer)

print(f'вычисления на {config.device}\n')
print('\n параметры запуска')
vars(config)

вычисления на cuda


 параметры запуска


{'raw_ds_path': './data/raw_dataset.txt',
 'ds_processed_path': './data/dataset_processed.txt',
 'ds_train_path': './data/train.csv',
 'ds_val_path': './data/val.csv',
 'ds_test_path': './data/test.csv',
 'model_path': './models/lstm_model.pth',
 'max_raw__ds_length': None,
 'batch_size': 256,
 'embedding_dim': 256,
 'hidden_dim': 128,
 'num_layers': 1,
 'dropout': 0.2,
 'learning_rate': 0.002,
 'num_epochs': 10,
 'max_length': 50,
 'train_val_split': 0.8,
 'val_rouge_samples': 200,
 'text_split_on_prediction': 0.75,
 'device': 'cuda',
 'tokenizer_name': 'distilgpt2',
 'pad_token_id': 50256,
 'pad_token': '<|endoftext|>',
 'vocab_size': 50257}

# Подготовка датасета

### 1. Получение и очистка исходных текстов

In [3]:
file = config.raw_ds_path
if not os.path.exists(file):
    !wget https://code.s3.yandex.net/deep-learning/tweets.txt -O {file}

In [ ]:
clean_corpus = process_dataset(config)

### 2. Разделение датасета на подсеты с сохранением в файлы 

In [ ]:
train_texts, val_texts, test_texts = split_dataset(texts=clean_corpus,config=config)

# Подготовка Dataloaders

### 1. Инициализация Datasets

In [ ]:
train_dataset = TextDataset(train_texts, tokenizer, config)
val_dataset = TextDataset(val_texts, tokenizer, config)
test_dataset = TextDataset(test_texts, tokenizer, config)

print(f'\ntrain_dataset len:', f"{len(train_dataset):_}")
print(f'val_dataset len:  ', f"{len(val_dataset):_}")
print(f'test_dataset len: ', f"{len(test_dataset):_}")

### 2. Инициализация Dataloaders

In [ ]:
dl_config = {
    'batch_size': config.batch_size,
    'num_workers': 4,
    'pin_memory': config.device=='cuda',
    'persistent_workers': True,
    'prefetch_factor': 2,
}

# подвыборка валидационного сета для расчета rouge на эпохах обучения
val_sampler = RandomSampler(
    val_dataset, 
    replacement=False, 
    num_samples=config.val_rouge_samples   
)

train_dataloader = DataLoader(train_dataset, **dl_config, shuffle=True)
val_dataloader = DataLoader(val_dataset, **dl_config, shuffle=False)
val_sample_dataloader = DataLoader(val_dataset, **dl_config, shuffle=False, sampler=val_sampler)
test_dataloader = DataLoader(test_dataset, **dl_config, shuffle=False)

print(f'батчей в train:    {len(train_dataloader)}')
print(f'батчей в val:      {len(val_dataloader)}')
print(f'батчей в val_smpl: {len(val_sample_dataloader)}')
print(f'батчей в test:     {len(test_dataloader)}')

# Обучение модели

### Инициализация модели

In [ ]:
model = LSTMModel(config)
print('количество параметров: ', f'{count_parameters(model):_}')
model = model.to(config.device)

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=config.pad_token_id)
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

### Обучение и валидация

In [ ]:
train_model(
    model,
    tokenizer,
    config,
    train_dataloader,
    val_dataloader,
    val_sample_dataloader,
    criterion,
    optimizer,
)

### Сохранение модели

In [ ]:
torch.save({
        'model_state_dict': model.state_dict(),
        'config': config,
    }, config.model_path
    )

### Загрузка модели

In [ ]:
add_safe_globals([Config])
checkpoint = torch.load(config.model_path, map_location=config.device)
model = LSTMModel(checkpoint["config"]).to(config.device)

model.load_state_dict(checkpoint["model_state_dict"])
model.eval()

### Генерация дополнения текстов

In [ ]:
samples = random.sample(test_texts, 6) # 6 примеров
generate_samples(model, tokenizer, config, samples)

### Метрики rouge модели на тестовой выборке

In [ ]:
print('----модель lstm---------')
print('\nколичество параметров: ', f"{count_parameters(model):_}")
print('\nтекстов в выборке: ', f"{len(test_texts):_}", '\n')

test_dataloader = DataLoader(test_dataset, **dl_config, shuffle=False)

r1, r2 = calculate_rouge_lstm(
    model, test_dataloader, tokenizer, config, prefix="test"
)
print(f"\ntest rouge1: {r1:.4f}, test rouge2: {r2:.4f}")
print('-'*40)

# Сравнение с трансформером и выводы

### Метрики rouge трансформера на тестовой выборке

In [ ]:
print('----модель distilgpt2---------')
generator = pipeline("text-generation", model="distilgpt2", device=config.device)

print('\nколичество параметров: ', f"{count_parameters(generator.model):_}")
print('\nтекстов в выборке: ', f"{len(test_texts):_}", '\n')

r1,r2 = calculate_rouge_transformer(generator, config, test_texts)

print(f"\ntest rouge1: {r1:.4f}, test rouge2: {r2:.4f}")
print('-'*40)